Install dependencies

In [17]:
%pip install opencv-python keras numpy pygame --user

Note: you may need to restart the kernel to use updated packages.


Import dependencies

In [18]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

Load Alarm

In [19]:
mixer.init()
alarm_sound = mixer.Sound('alarm.wav')

Load Haar Cascades XML files

In [20]:
face_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
left_eye_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
right_eye_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')

<>:1: SyntaxWarning: invalid escape sequence '\h'
<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:1: SyntaxWarning: invalid escape sequence '\h'
<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
C:\Users\mail\AppData\Local\Temp\ipykernel_4068\3707589512.py:1: SyntaxWarning: invalid escape sequence '\h'
  face_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
C:\Users\mail\AppData\Local\Temp\ipykernel_4068\3707589512.py:2: SyntaxWarning: invalid escape sequence '\h'
  left_eye_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
C:\Users\mail\AppData\Local\Temp\ipykernel_4068\3707589512.py:3: SyntaxWarning: invalid escape sequence '\h'
  right_eye_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')


Load Custom Keras Model

In [21]:
model = load_model('drowsiness.keras')

Initializee video feed

In [22]:
capture = cv2.VideoCapture(0)

if capture.isOpened():
    capture = cv2.VideoCapture(0)
if not capture.isOpened():
    raise IOError("Cannot open webcam")

Declare variables

In [23]:
labels = ['Close','Open']
path = os.getcwd()
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
counter = 0
time = 0
thick = 2
right_eye_pred=[99]
left_eye_pred=[99]
face_pred=[99]

Perform Detection

In [24]:
while(True):
    ret, frame = capture.read()
    height,width = frame.shape[:2] 

    #convert the captured image to grey color:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #perform detection(this will return x,y coordinates , height , width of the boundary boxes object)
    faces = face_detection.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = left_eye_detection.detectMultiScale(gray)
    right_eye =   right_eye_detection.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (100,height) , (0,0,0) , thickness=cv2.FILLED )
    cv2.rectangle(frame, (290,height-50) , (540,height) , (0,0,0) , thickness=cv2.FILLED )

    #iterating over faces and drawing boundary boxes for each face:
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )
        
    #yawn detection
    for (x,y,w,h) in faces:
        face = frame[y:y+h,x:x+w]
        counter += 1
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        face = cv2.resize(face, (24,24))
        face = face/255
        face = face.reshape(24,24,-1)
        face = np.expand_dims(face,axis=0)
        face_pred = np.argmax(model.predict(face), axis=1)
        if(face_pred[0] == 2):
            labels = 'Yawn'
        if(face_pred[0] == 3):
            labels = 'No_yawn'
        break
        
    #iterating over right eye:
    for (x,y,w,h) in right_eye:
        #pull out the right eye image from the frame:
        right_one=frame[y:y+h,x:x+w]
        counter += 1
        right_one = cv2.cvtColor(right_one,cv2.COLOR_BGR2GRAY)
        right_one = cv2.resize(right_one,(24,24))
        right_one = right_one/255
        right_one =  right_one.reshape(24,24,-1)
        right_one = np.expand_dims(right_one,axis=0)
        right_eye_pred = np.argmax(model.predict(right_one), axis=1)
        if(right_eye_pred[0] == 1):
            labels = 'Open' 
        if(right_eye_pred[0]==0):
            labels = 'Closed'
        break

    #iterating over left eye:
    for (x,y,w,h) in left_eye:
        #pull out the left eye image from the frame:
        left_one=frame[y:y+h,x:x+w]
        counter += 1
        left_one = cv2.cvtColor(left_one,cv2.COLOR_BGR2GRAY)  
        left_one = cv2.resize(left_one,(24,24))
        left_one = left_one/255
        left_one = left_one.reshape(24,24,-1)
        left_one = np.expand_dims(left_one,axis=0)
        left_eye_pred = np.argmax(model.predict(left_one), axis=1)
        if(left_eye_pred[0] == 1):
            labels ='Open'   
        if(left_eye_pred[0] == 0):
            labels ='Closed'
        break

    if(right_eye_pred[0] == 0 and left_eye_pred == 0 or face_pred == 2):
        time += 1
        cv2.putText(frame,"Drowsy",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(right_eye_pred[0]==1 or left_eye_pred[0]==1):
    else:
        time -= 1
        cv2.putText(frame,"Alert",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        
    
        
    if(time<0):
        time=0   
    cv2.putText(frame,'Wake up Time !!:'+str(time),(300,height-20), font, 1,(0,0,255),1,cv2.LINE_AA)
    if(time>10):
        #person is feeling dazzi we will alert :
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            alarm_sound.play()
            
        except:  # isplaying = False
            pass
        if(thick < 16):
            thick = thick+2
        else:
            thick=thick-2
            if(thick<2):
                thick=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thick)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
capture.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━